<a href="https://colab.research.google.com/github/gpricechristie/hsstWebServiceDemo/blob/main/dynamicWebsite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dynamic database backed website
As for the previous example,this code uses flask to create a website thats rendered through the use of html templates. However, this site also uses a database (in this case an SQLite database) to make the website content dynamic.

As in previous examples we also use pyngrok to expose the locally hosted web service to the public internet



In [ ]:
!pip install pyngrok
from flask import Flask, render_template, request, redirect, g
from pyngrok import ngrok
import sqlite3
import os

##Create the website database
Use the sqlite3 package to create a new database (need the parent directory to exist first) and connect to it

In [ ]:
if not os.path.exists('db'):
  os.makedirs('db')
conn = sqlite3.connect("db/test.db")

Create a new table that will hold the data the website reads while rendering. Our database will contain a users first and last name and their thoughts on how interesting clinical computing is

In [ ]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS computing_data;")
cursor.execute("CREATE TABLE IF NOT EXISTS computing_data ( fname TEXT, lname TEXT, valueofcomputing TEXT );")

##Boilerplate code to build website
As for the StaticWebsite notebook we need to create the flask server object, and authenticate and create the ngrok tunnel

In [ ]:
app = Flask(__name__)

ngrok.set_auth_token("2OxbwWyt7aHTrLuvIkLgdvgmSLA_81zjJCaMLzNbzx89uDb8r")
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, "5000"))

##Functions to render the site
In this example we use a series of functions that are called at different times during the rendering of the website

This function uses the before_request decorator and is called before each web request is processed. It connects to the database we previously created. The 'g' indicates the database connection is global (ie visible to the application outside of this specific function)

In [ ]:
@app.before_request
def before_request():
    g.db = sqlite3.connect("db/test.db")

This function is called after each request andcloses the database connection

In [ ]:
@app.teardown_request
def teardown_request(exception):
    if hasattr(g, 'db'):
        g.db.close()

This function is called when the URL /add_data is called. Rather than render a webpage however (as in the previous examples) it calls some code that inserts data supplied as part of the URL into the database.

The webform.html template uses some html code to create a webform. When that form is used it sends an http POST request to the /add_data page which calls the below function. At the end of the function the request is redirected to the root page which renders the site.

In [ ]:
@app.route('/add_data', methods = ['POST'])
def add_data():
    fname = request.form['firstname']
    lname = request.form['lastname']
    valueof = request.form['valueofcomputing']

    g.db.execute("INSERT INTO computing_data(fname, lname, valueofcomputing) VALUES (?,?,?)", [fname,lname,valueof])
    g.db.commit()

    return redirect('/')



The main rendering function renders the webform.html. The difference to the previous example is that the function queries the database to get a copy of all the data contained in it and builds these into an array of strings. These are passed as arguments to the render_template() function which inserts them into the html rendered.


In [ ]:
@app.route('/')
def write_dynamic_page():
    data=g.db.execute("SELECT * FROM computing_data").fetchall()
    displayComments=[]
    for entry in data:
        displayComments.append(entry[0]+" "+entry[1]+" thinks that clinical computing is "+entry[2])

    return render_template('webform.html',len = len(displayComments),displayComments=displayComments)

##Run application
Load the html template and images to local file system. 

In [ ]:
!mkdir templates
!mkdir static
!wget https://raw.githubusercontent.com/gpricechristie/hsstWebServiceDemo/main/templates/webform.html -P ./templates
!wget https://raw.githubusercontent.com/gpricechristie/hsstWebServiceDemo/main/static/christie.jpg -P ./static

Run the application to create the local webserver. Click on the ngrok.io address from above to visit the page you have just created.

In [ ]:
if __name__ == "__main__":
      app.run()